In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = (SparkSession
 .builder
 .appName("Don Quijote")
 .getOrCreate())

#### a. Descargar el Quijote
https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79


In [12]:
# !wget "https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt"
df = spark.read.option("header", "false").csv("el_quijote.txt")

Aplicar no solo count (para obtener el número de líneas) y show sino probar distintas
sobrecargas del método show (con/sin truncate, indicando/sin indicar num de filas,
etc) así como también los métodos, head, take, first (diferencias entre estos 3?)

In [13]:
df.count()

2185

In [25]:
df.show(3,truncate = False)

+----------------------------+
|_c0                         |
+----------------------------+
|DON QUIJOTE DE LA MANCHA    |
|Miguel de Cervantes Saavedra|
|PRIMERA PARTE               |
+----------------------------+
only showing top 3 rows



In [18]:
df.head(3)

[Row(_c0='DON QUIJOTE DE LA MANCHA'),
 Row(_c0='Miguel de Cervantes Saavedra'),
 Row(_c0='PRIMERA PARTE')]

In [19]:
df.take(3)

[Row(_c0='DON QUIJOTE DE LA MANCHA'),
 Row(_c0='Miguel de Cervantes Saavedra'),
 Row(_c0='PRIMERA PARTE')]

In [22]:
df.first()

Row(_c0='DON QUIJOTE DE LA MANCHA')

#### b. Del ejercicio de M&M aplicar:

In [3]:
mnm_file = "mnm_dataset.csv"
mnm_df = spark\
            .read\
            .option("header", "true")\
            .option("inferSchema", "true")\
            .csv(mnm_file)
mnm_df.show(5)

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   10|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
+-----+------+-----+
only showing top 5 rows



i. Otras operaciones de agregación como el Max con otro tipo de ordenamiento
(descendiente).

In [4]:
max_mnm_df = (mnm_df
 .select("State", "Color", "Count")
 .groupBy("State", "Color")
 .agg(F.max("Count").alias("Maximo"))
 .orderBy("Maximo", ascending=False))
max_mnm_df.show(5)

+-----+------+------+
|State| Color|Maximo|
+-----+------+------+
|   AZ| Green|   100|
|   CA|   Red|   100|
|   WY| Green|   100|
|   NM| Green|   100|
|   WA|Orange|   100|
+-----+------+------+
only showing top 5 rows



ii. hacer un ejercicio como el “where” de CA que aparece en el libro pero
indicando más opciones de estados (p.e. NV, TX, CA, CO).

In [18]:
nv_co_count_mnm_df = (mnm_df
 .select("State", "Color", "Count")
 .where((F.col("State") == "NV") | (F.col("State") == "CO"))
 .groupBy("State", "Color")
 .agg(F.count("Count").alias("Total"))
 .orderBy("Total", ascending=False))
nv_co_count_mnm_df.toPandas()

,State,Color,Total
0,CO,Yellow,1721
1,CO,Green,1713
2,NV,Orange,1712
3,NV,Green,1698
4,CO,Blue,1695
5,NV,Yellow,1675
6,NV,Blue,1673
7,NV,Brown,1657
8,CO,Brown,1656
9,CO,Orange,1642


In [17]:
nv_co_count_mnm_df = mnm_df\
                         .select("State", "Color")\
                         .where(
                            (F.col("State") == "NV") | (F.col("State") == "CO")
                         )\
                         .groupBy("State")\
                         .pivot("Color")\
                         .count()

nv_co_count_mnm_df.toPandas()

,State,Blue,Brown,Green,Orange,Red,Yellow
0,NV,1673,1657,1698,1712,1610,1675
1,CO,1695,1656,1713,1642,1624,1721


iii. Hacer un ejercicio donde se calculen en una misma operación el Max, Min,
Avg, Count. Revisar el API (documentación) donde encontrarán este ejemplo:
ds.agg(max($"age"), avg($"salary"))
ds.groupBy().agg(max($"age"), avg($"salary"))
NOTA: $ es un alias de col()

In [21]:
op_mnm_df = (mnm_df
                   .select("State", "Color", "Count")
                   .groupBy("State", "Color")
                   .agg(
                         F.count("Count").alias("Total"),
                         F.max("Count").alias("Max"),
                         F.min("Count").alias("Min"),
                         F.round(F.avg("Count"),3).alias("Avg")
                       )
                )
# .orderBy("Total", ascending=False))
op_mnm_df.toPandas().head()

,State,Color,Total,Max,Min,Avg
0,WY,Green,1695,100,10,55.657
1,NV,Red,1610,100,10,55.494
2,UT,Blue,1655,100,10,54.367
3,WA,Orange,1658,100,10,55.200
4,NM,Green,1682,100,10,54.197


iv. Hacer también ejercicios en SQL creando tmpView

In [24]:
mnm_df.createOrReplaceTempView("mnm")
spark.sql("""SELECT * FROM mnm""").show(5)

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   10|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
+-----+------+-----+
only showing top 5 rows



In [181]:
spark.sql("""SELECT State,
                    Color,
                    count(Count) as Total,
                    max(Count) as Maximo,
                    min(Count) as Minimo,
                    round(avg(Count),3) as Avg
             FROM mnm
             GROUP BY State,Color
              """).show()

+-----+------+-----+------+------+------+
|State| Color|Total|Maximo|Minimo|   Avg|
+-----+------+-----+------+------+------+
|   WY| Green| 1695|   100|    10|55.657|
|   NV|   Red| 1610|   100|    10|55.494|
|   UT|  Blue| 1655|   100|    10|54.367|
|   WA|Orange| 1658|   100|    10|  55.2|
|   NM| Green| 1682|   100|    10|54.197|
|   CA|  Blue| 1603|   100|    10|55.598|
|   WA|   Red| 1671|   100|    10|55.854|
|   NV| Brown| 1657|   100|    10|55.811|
|   AZ| Green| 1676|   100|    10|54.822|
|   CA|   Red| 1656|   100|    10| 55.27|
|   AZ|Orange| 1689|   100|    10|54.283|
|   CO|  Blue| 1695|   100|    10| 55.11|
|   NM|Orange| 1665|   100|    10|54.805|
|   NM|Yellow| 1688|   100|    10|54.945|
|   WY|Orange| 1595|   100|    10|55.145|
|   UT|Yellow| 1645|   100|    10|54.264|
|   WY|   Red| 1670|   100|    10|54.951|
|   OR|  Blue| 1646|   100|    10|54.998|
|   NV|Orange| 1712|   100|    10|54.865|
|   AZ|Yellow| 1654|   100|    10|54.985|
+-----+------+-----+------+------+

In [177]:
spark.sql("""SELECT State,Color,COUNT(*) as Cantidad
             FROM mnm  
             WHERE State = "TX" 
             GROUP BY State,Color
             ORDER BY Cantidad DESC
             """).show()

+-----+------+--------+
|State| Color|Cantidad|
+-----+------+--------+
|   TX| Green|    1737|
|   TX|   Red|    1725|
|   TX|Yellow|    1703|
|   TX|Orange|    1652|
|   TX| Brown|    1641|
|   TX|  Blue|    1614|
+-----+------+--------+

